In [ ]:
%pip install gdown
import gdown

url = "https://drive.google.com/drive/folders/1I34vsWKtKYZTyxsaHSx3n-zzAmJaOpf4?usp=share_link"
gdown.download_folder(url, quiet=True)

In [ ]:
!pip install tqdm
!pip install dlib

In [8]:
import os
import zipfile
import cv2
import numpy as np
import shutil

def weighted_average_dimensions(folder):
  total_weight, weighted_sum_h, weighted_sum_w = 0, 0, 0
  for file_name in os.listdir(folder):
      file_path = os.path.join(folder, file_name)
      image = cv2.imread(file_path)
      height, width = image.shape[:2]
      weight = height * width
      total_weight += weight
      weighted_sum_h += height * weight
      weighted_sum_w += width * weight
  average_height = weighted_sum_h / total_weight
  average_width = weighted_sum_w / total_weight

  return int(average_height), int(average_width)

def resize_images(feature, folder, target_size):
  output_folder = f"resized_{feature}"
  os.makedirs(output_folder, exist_ok=True)
  for file_name in os.listdir(folder):
    file_path = os.path.join(folder, file_name)
    image = cv2.imread(file_path)
    resized_image = cv2.resize(image, target_size)
    output_path = os.path.join(output_folder, file_name)
    cv2.imwrite(output_path, resized_image)

def main():
  folder_path = "/content/68_landmarks"
  for file_name in os.listdir(folder_path):

    # zipped files in the folder biometric_systems_dataset
    if file_name.endswith(".zip"):

      # full path to the zip file
      file_path = os.path.join(folder_path, file_name)

      # create folder in the same path with the name of the zip file
      extract_folder = os.path.join(folder_path, file_name.replace(".zip", ""))
      os.makedirs(extract_folder, exist_ok=True)

      # open and extract the file
      with zipfile.ZipFile(file_path, mode="r") as zip_ref:
        zip_ref.extractall(extract_folder)

  features = ["eyes", "mouth", "nose"]
  for feature in features:
    feature_folder_path = os.path.join(folder_path, feature)
    average_height, average_width = weighted_average_dimensions(feature_folder_path)
    resize_images(feature, feature_folder_path, (average_width, average_height))
    shutil.make_archive(f"resized_{feature}", "zip",f"resized_{feature}")


if __name__ == "__main__":
  main()